> Esse notebook faz referências ao curso gratuíto 'Intro to TensorFlow for Deep Learning' da UDACITY (https://classroom.udacity.com/courses/ud187/)

# Retreinando classificador de imagens com TF2 e TF Hub

<table align="left">
<td align="center">
  <a target="_blank"  href="https://colab.research.google.com/github/dntxos/TFKIT201/blob/master/Notebooks/Retreinando_classificador_de_imagens_com_TF2_e_TF_Hub.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /><br>Run in Google Colab
  </a>
</td>
<td align="center">
  <a target="_blank"  href="https://github.com/dntxos/TFKIT201/blob/master/Notebooks/Retreinando_classificador_de_imagens_com_TF2_e_TF_Hub.ipynb">
    <img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /><br>View source on GitHub</a>
</td>
</table>

## Introdução

Modelos classificadores de imagens possuem milhões de parâmetros. Treinar um modelo a partir
do ZERO requer um enorme volume de dados anotados e muito poder computacional. Transferência de aprendizado (transfer learning)
é uma técnica que permite encurtar esse caminho utilizando modelos já treinados como Inception ou MobileNet reaproveitando
todo o aprendizado e poder computacional gasto para treinar esses modelos, treinando apenas as últimas camadas da rede.

Video explicativo do processo: https://youtu.be/JhR1_WZCj54

### Procurando pela ferramenta para criar um modelo classificador de imagens?

Este é um notebook tutorial. Se você quer uma ferramenta que apenas compila um modelo TensorFlow ou TF Lite, dê uma olhada na ferramenta de linha de comando [make_image_classifier](https://github.com/tensorflow/hub/tree/master/tensorflow_hub/tools/make_image_classifier), que pode ser [instalada](https://www.tensorflow.org/hub/installation) através do PIP `tensorflow-hub[make_image_classifier]`.


## Configurando TensorFlow 2 e outras bibliotecas

In [0]:
# Desconsidere esse bloco no caso de executar em 'Colab Hosted Runtime'!

# Considerando que o container seja baseado na imagem docker do Tensorflow 2.0.1, os pacotes abaixo precisam ser instalados:

!pip -q --disable-pip-version-check install pillow
!pip -q --disable-pip-version-check install scipy
!pip -q --disable-pip-version-check install tensorflow_hub

# Caso tenha problemas na importação das bibliotecas, reinicie o Kernel.

In [0]:
try:
  # %tensorflow_version EXISTE SOMENTE NO COLAB
  %tensorflow_version 2.x
except Exception:
  pass

In [0]:
import itertools
import os

import matplotlib.pylab as plt
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub

print("Versão do TensorFlow:", tf.__version__)
print("Versão do TF Hub:", hub.__version__)
print("GPU", "DISPONÍVEL" if len(tf.config.list_physical_devices('GPU')) else "NÃO DISPONÍVEL")

## Selecione o módulo (TF2 SavedModel) para utilizar no retreino

Para iniciantes, use https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4. Você também pode acessar essa mesma URL para consultar a documentação do modelo no TFHUB. (`hub.Modules` para TF 1.x não funciona aqui.)

In [0]:
#@markdown Escolha um dos modelos da lista abaixo, ou entre com o nome do modelo `image-feature-vector` disponível para TF2 na página do TFHUB (https://tfhub.dev/s?module-type=image-feature-vector&tf-version=tf2).
module_selection = ("mobilenet_v2_100_224", 224) #@param ["(\"mobilenet_v2_100_224\", 224)", "(\"inception_v3\", 299)"] {type:"raw", allow-input: true}
handle_base, pixels = module_selection
MODULE_HANDLE ="https://tfhub.dev/google/imagenet/{}/feature_vector/4".format(handle_base)
IMAGE_SIZE = (pixels, pixels)
print("Utilizando {} com tamanho de entrada {} pixels".format(MODULE_HANDLE, IMAGE_SIZE))

#@markdown Defina o tamanho do lote de imagens para o treinamento. Quanto maior o tamanho do lote, mais memória será demandada ao treinamento.
BATCH_SIZE = 32 #@param {type:"integer"}

## Configurando o Dataset (Flowers)

As entradas são redimensionadas adequadamente para o módulo selecionado. E aumento do conjunto de dados (*Data augmentation, ou seja, distorções aleatórias de uma imagem cada vez que é lida) melhora o treinamento.

In [0]:
data_dir = tf.keras.utils.get_file(
    'flower_photos',
    'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
    untar=True)

In [0]:
datagen_kwargs = dict(rescale=1./255, validation_split=.20)
dataflow_kwargs = dict(target_size=IMAGE_SIZE, batch_size=BATCH_SIZE,
                   interpolation="bilinear")

valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    **datagen_kwargs)
valid_generator = valid_datagen.flow_from_directory(
    data_dir, subset="validation", shuffle=False, **dataflow_kwargs)

#@markdown Ative `do_data_augmentation`para aumentar artificialmente o número de imagens em nosso conjunto de treinamento aplicando transformações aleatórias de imagem nas imagens existentes no conjunto de treinamento. É muito útil para datasets muito pequenos e ajuda a generalizar. (Video: https://youtu.be/Qgd7maIVytI)
do_data_augmentation = False #@param {type:"boolean"}
if do_data_augmentation:
  train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
      rotation_range=40,
      horizontal_flip=True,
      width_shift_range=0.2, height_shift_range=0.2,
      shear_range=0.2, zoom_range=0.2,
      **datagen_kwargs)
else:
  train_datagen = valid_datagen
train_generator = train_datagen.flow_from_directory(
    data_dir, subset="training", shuffle=True, **dataflow_kwargs)


## Definindo o modelo

Basta colocar um classificador linear em cima do `feature_extractor_layer` com módulo do TFHUB.

Para mais velocidade, começamos com um `feature_extractor_layer` não treinável, mas você também pode ativar o ajuste fino (fine-tuning) para obter maior precisão.

In [0]:
do_fine_tuning = False #@param {type:"boolean"}

In [0]:
print("Compilando o modelo com", MODULE_HANDLE)
model = tf.keras.Sequential([
    hub.KerasLayer(MODULE_HANDLE, trainable=do_fine_tuning),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(train_generator.num_classes, activation='softmax',
                          kernel_regularizer=tf.keras.regularizers.l2(0.0001))
])
model.build((None,)+IMAGE_SIZE+(3,))
model.summary()

## Treinando o modelo

In [0]:
model.compile(
  optimizer=tf.keras.optimizers.SGD(lr=0.005, momentum=0.9), 
  loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
  metrics=['accuracy'])

In [0]:
steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = valid_generator.samples // valid_generator.batch_size
hist = model.fit_generator(
    train_generator,
    epochs=5, steps_per_epoch=steps_per_epoch,
    validation_data=valid_generator,
    validation_steps=validation_steps).history

In [0]:
plt.figure()
plt.ylabel("Loss (Treinamento e validação)")
plt.xlabel("Passos do treinamento")
plt.ylim([0,2])
plt.plot(hist["loss"])
plt.plot(hist["val_loss"])

plt.figure()
plt.ylabel("Acurácia (Treinamento e validação)")
plt.xlabel("Passos do treinamento")
plt.ylim([0,1])
plt.plot(hist["accuracy"])
plt.plot(hist["val_accuracy"])

Finalmente, seu modelo pode ser salvo para implantação no TFX/Tensorflow_Serving ou para dispositivos móveis com TF Lite a seguir.

Visualizações da sua rede pode te ajudar a entender ainda mais (https://towardsdatascience.com/understanding-your-convolution-network-with-visualizations-a4883441533b)

In [0]:
saved_model_path = "/tmp/saved_flowers_model"
tf.saved_model.save(model, saved_model_path)

## Opcional: Implantação em TensorFlow Lite

[TensorFlow Lite](https://www.tensorflow.org/lite) permite implantar modelos TensorFlow em dispositivos móveis e IoT. O código abaixo demonstra como converter o modelo treinado para o formato TF Lite e utilizar ferramentas pós-treinamento do [Tensorflow Model Optimization Toolkit](https://www.tensorflow.org/model_optimization). Ao final, executamos no 'Interpretador TF Lite' para examinar a qualidade dos resultados

  * A conversão sem otimização fornece os mesmos resultados de antes (até o erro de arredondamento).
  * A conversão com otimização sem dados quantifica os pesos do modelo para 8 bits, mas a inferência ainda usa computação em ponto flutuante para as ativações da rede neural. Isso reduz o tamanho do modelo quase por um fator de 4 e melhora a latência da CPU em dispositivos móveis.
  * Além disso, o cálculo das ativações da rede neural pode ser quantizado para números inteiros de 8 bits, se um pequeno conjunto de dados de referência for fornecido para calibrar o intervalo de quantização. Em um dispositivo móvel, isso acelera ainda mais a inferência e possibilita a execução em aceleradores como o EdgeTPU.

In [0]:
#@title Configurações da otimização
optimize_lite_model = False  #@param {type:"boolean"}
#@markdown Definir um valor maior que zero permite a quantização de ativações de redes neurais. Algumas dezenas já são uma quantia útil.
num_calibration_examples = 60  #@param {type:"slider", min:0, max:1000, step:1}
representative_dataset = None
if optimize_lite_model and num_calibration_examples:
  # Use a bounded number of training examples without labels for calibration.
  # TFLiteConverter expects a list of input tensors, each with batch size 1.
  representative_dataset = lambda: itertools.islice(
      ([image[None, ...]] for batch, _ in train_generator for image in batch),
      num_calibration_examples)

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_path)
if optimize_lite_model:
  converter.optimizations = [tf.lite.Optimize.DEFAULT]
  if representative_dataset:  # This is optional, see above.
    converter.representative_dataset = representative_dataset
lite_model_content = converter.convert()

with open("/tmp/lite_flowers_model", "wb") as f:
  f.write(lite_model_content)
print("Wrote %sTFLite model of %d bytes." %
      ("optimized " if optimize_lite_model else "", len(lite_model_content)))

In [0]:
interpreter = tf.lite.Interpreter(model_content=lite_model_content)
# This little helper wraps the TF Lite interpreter as a numpy-to-numpy function.
def lite_model(images):
  interpreter.allocate_tensors()
  interpreter.set_tensor(interpreter.get_input_details()[0]['index'], images)
  interpreter.invoke()
  return interpreter.get_tensor(interpreter.get_output_details()[0]['index'])

In [0]:
#@markdown Para experimentação rápida, comece com um número moderado de exemplos.
num_eval_examples = 50  #@param {type:"slider", min:0, max:700}
eval_dataset = ((image, label)  # TFLite expects batch size 1.
                for batch in train_generator
                for (image, label) in zip(*batch))
count = 0
count_lite_tf_agree = 0
count_lite_correct = 0
for image, label in eval_dataset:
  probs_lite = lite_model(image[None, ...])[0]
  probs_tf = model(image[None, ...]).numpy()[0]
  y_lite = np.argmax(probs_lite)
  y_tf = np.argmax(probs_tf)
  y_true = np.argmax(label)
  count +=1
  if y_lite == y_tf: count_lite_tf_agree += 1
  if y_lite == y_true: count_lite_correct += 1
  if count >= num_eval_examples: break
print("TF Lite model agrees with original model on %d of %d examples (%g%%)." %
      (count_lite_tf_agree, count, 100.0 * count_lite_tf_agree / count))
print("TF Lite model is accurate on %d of %d examples (%g%%)." %
      (count_lite_correct, count, 100.0 * count_lite_correct / count))



---



Este notebook é baseado no [TF Hub for TF2: Retraining an image classifier](https://github.com/tensorflow/hub/blob/master/examples/colab/tf2_image_retraining.ipynb), adaptado para rodar localmente, sem código deprecado, traduzido para o português com links para vídeos do curso gratuíto 'ud187 - Intro to TensorFlow for Deep Learning' da UDACITY (https://classroom.udacity.com/courses/ud187/)

##### Copyright 2019 The TensorFlow Hub Authors.
Licensed under the Apache License, Version 2.0 (the "License");

In [0]:
# Copyright 2019 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================